**Exercise 1.1**
The general optimization problem for the given problem statement is:\
$ min \sum_{w=1}^8\sum_{m=1}^8 C(w,m)*x_{wm} $\
$s.t. \sum_{w=1}^8x_{wm}=d_m ∀mϵ${ 1,2,....8 }\
$ \sum_{m=1}^8x_{wm} <= s_w ∀wϵ${1,2,....8}\
$ x_{wm}>= 0 ∀w,m ϵ$ {1,2,....,8}\
where C(w,m) represents the cost of transporting one truck load of EV from $w^{th}$ warehouse to $m^{th}$ market, $d_m$ represents the demand of each market m , $s_w$ represents the available stock of warehouse w.
Our decision variables are $x_{wm}$ which denote the number of truckloads of EV transported from warehouse 'w' to market 'm'.
We have one more constraint of non negativity as the no. of truckloads transported can not take negative values.


In [ ]:
!pip install -q pyomo

In [ ]:
from pyomo.environ import *

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('lab6_ex1.csv')

In [ ]:
df.head(5)

Warehouses  Hassan  Jodhpur  Trichy  Simla  Vijayawada  Dibrugarh  Raipur  \
0  Ahmedabad     827      817    1470   2272        1615       3143    1264   
1  Bengaluru     559     1832     751   3772         637       3773    1856   
2    Chennai    1109     2443     577   3865         798       3521    1984   
3      Delhi    2943     1471    4676   1337        2702       2741    2482   
4    Kolkata    3742     3089    2931   3199        2226       1739    1691   

   Agartala  Avail. Stock  
0      4465           350  
1      5459           185  
2      5303           305  
3      4148           180  
4      2128           205

In [ ]:
#number of warehouses
M=len(df.index)-1
#number of markets
N=len(df.columns)-2

In [ ]:
model=ConcreteModel()

In [ ]:
row_indices= range(M)
col_indices = range(N)

In [ ]:
model.x=Var(row_indices,col_indices,domain=NonNegativeReals)

In [ ]:
model.objective=Objective(expr=sum(model.x[i,j]*df.iloc[i,j+1] for i in row_indices for j in col_indices),sense=minimize)

In [ ]:
model.constraint=ConstraintList()

In [ ]:
for i in row_indices:
  model.constraint.add(expr=sum(model.x[i,j] for j in col_indices)<= df.iloc[i,-1])

for j in col_indices:
  model.constraint.add(expr=sum(model.x[i,j] for i in row_indices)==df.iloc[-1,j+1])

In [ ]:
model.pprint()

4 Set Declarations
    constraint_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   16 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :   64 : {(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)}
    x_index_0 : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
     

In [ ]:
!apt-get install -y -qq coinor-cbc

In [ ]:
opt_cbc = SolverFactory('cbc')
result = opt_cbc.solve(model)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [ ]:
#display solution
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in row_indices:
  for j in col_indices:
    if(model.x[i,j].value != 0):
      print('number of truck-loads between ',df.iloc[i,0],' and ', df.columns[j+1],':', model.x[i,j].value)

print('\nConstraints')
model.constraint.display()


Objective =  2192870.0

Decision Variables
number of truck-loads between  Ahmedabad  and  Jodhpur : 300.0
number of truck-loads between  Ahmedabad  and  Dibrugarh : 45.0
number of truck-loads between  Ahmedabad  and  Raipur : 5.0
number of truck-loads between  Bengaluru  and  Trichy : 120.0
number of truck-loads between  Bengaluru  and  Vijayawada : 65.0
number of truck-loads between  Chennai  and  Trichy : 305.0
number of truck-loads between  Delhi  and  Simla : 95.0
number of truck-loads between  Delhi  and  Dibrugarh : 85.0
number of truck-loads between  Kolkata  and  Dibrugarh : 95.0
number of truck-loads between  Kolkata  and  Agartala : 110.0
number of truck-loads between  Lucknow  and  Simla : 175.0
number of truck-loads between  Mumbai  and  Hassan : 205.0
number of truck-loads between  Mumbai  and  Raipur : 120.0
number of truck-loads between  Hyderabad  and  Vijayawada : 310.0
number of truck-loads between  Hyderabad  and  Raipur : 60.0

Constraints
constraint : Size=16
    

**Exercise 1.7**
1. number of truck-loads between  Ahmedabad  and  Jodhpur : 300.0
2. number of truck-loads between  Ahmedabad  and  Dibrugarh : 45.0
3. number of truck-loads between  Ahmedabad  and  Raipur : 5.0
4. number of truck-loads between  Bengaluru  and  Trichy : 120.0
5. number of truck-loads between  Bengaluru  and  Vijayawada : 65.0
6. number of truck-loads between  Chennai  and  Trichy : 305.0
7. number of truck-loads between  Delhi  and  Simla : 95.0
8. number of truck-loads between  Delhi  and  Dibrugarh : 85.0
9. number of truck-loads between  Kolkata  and  Dibrugarh : 95.0
10. number of truck-loads between  Kolkata  and  Agartala : 110.0
11. number of truck-loads between  Lucknow  and  Simla : 175.0
12. number of truck-loads between  Mumbai  and  Hassan : 205.0
13. number of truck-loads between  Mumbai  and  Raipur : 120.0
14. number of truck-loads between  Hyderabad  and  Vijayawada : 310.0
15. number of truck-loads between  Hyderabad  and  Raipur : 60.0\
The total cost of transporting truckloads of EVs is **2192870.0**

**Exercise 1.8**
If the Lucknow-Simla link is disrupted and no transportation is possible on the Lucknow-Simla links, to solve the optimization model without changing the pyomo model we should increase the cost of transportation for this link to a very high amount as this would result in the model not taking up this route for calculating the optimal value since our objective is to minimize the transportation cost and including this link in the model will result in very high cost.

In [ ]:
df.iloc[5,4]=15000

In [ ]:
df.head(10)

Warehouses  Hassan  Jodhpur  Trichy  Simla  Vijayawada  Dibrugarh  Raipur  \
0  Ahmedabad     827      817    1470   2272        1615       3143    1264   
1  Bengaluru     559     1832     751   3772         637       3773    1856   
2    Chennai    1109     2443     577   3865         798       3521    1984   
3      Delhi    2943     1471    4676   1337        2702       2741    2482   
4    Kolkata    3742     3089    2931   3199        2226       1739    1691   
5    Lucknow    2353     1160    3295  15000        2369       2365    1293   
6     Mumbai    1356     1617    2045   2941        2334       4151    1937   
7  Hyderabad     195     1862    1331   2921         402       2825     563   
8     Demand     205      300     425    270         375        225     185   

   Agartala  Avail. Stock  
0      4465           350  
1      5459           185  
2      5303           305  
3      4148           180  
4      2128           205  
5      3642           175  
6      4287           325  
7      4441           370  
8       110          2095

In [ ]:
model.objective.deactivate()
model.new_objective=Objective(expr=sum(model.x[i,j]*df.iloc[i,j+1] for i in row_indices for j in col_indices),sense=minimize)

In [ ]:
opt_cbc = SolverFactory('cbc')
result = opt_cbc.solve(model)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [ ]:
#display solution
print('\nObjective = ', model.new_objective())

print('\nDecision Variables')
for i in row_indices:
  for j in col_indices:
    if(model.x[i,j].value !=0):
      print('number of truck-loads between ',df.iloc[i,0],' and ', df.columns[j+1],':', model.x[i,j].value)

print('\nConstraints')
model.constraint.display()


Objective =  2380570.0

Decision Variables
number of truck-loads between  Ahmedabad  and  Jodhpur : 300.0
number of truck-loads between  Ahmedabad  and  Raipur : 50.0
number of truck-loads between  Bengaluru  and  Trichy : 120.0
number of truck-loads between  Bengaluru  and  Vijayawada : 65.0
number of truck-loads between  Chennai  and  Trichy : 305.0
number of truck-loads between  Delhi  and  Simla : 180.0
number of truck-loads between  Kolkata  and  Dibrugarh : 95.0
number of truck-loads between  Kolkata  and  Agartala : 110.0
number of truck-loads between  Lucknow  and  Dibrugarh : 130.0
number of truck-loads between  Lucknow  and  Raipur : 45.0
number of truck-loads between  Mumbai  and  Hassan : 205.0
number of truck-loads between  Mumbai  and  Simla : 90.0
number of truck-loads between  Mumbai  and  Raipur : 30.0
number of truck-loads between  Hyderabad  and  Vijayawada : 310.0
number of truck-loads between  Hyderabad  and  Raipur : 60.0

Constraints
constraint : Size=16
    Key

**Exercise 1.9**
The new solution values are:
1. number of truck-loads between  Ahmedabad  and  Jodhpur : 300.0
2. number of truck-loads between  Ahmedabad  and  Raipur : 50.0
3. number of truck-loads between  Bengaluru  and  Trichy : 120.0
4. number of truck-loads between  Bengaluru  and  Vijayawada : 65.0
5. number of truck-loads between  Chennai  and  Trichy : 305.0
6. number of truck-loads between  Delhi  and  Simla : 180.0
7. number of truck-loads between  Kolkata  and  Dibrugarh : 95.0
8. number of truck-loads between  Kolkata  and  Agartala : 110.0
9. number of truck-loads between  Lucknow  and  Dibrugarh : 130.0
10. number of truck-loads between  Lucknow  and  Raipur : 45.0
11. number of truck-loads between  Mumbai  and  Hassan : 205.0
12. number of truck-loads between  Mumbai  and  Simla : 90.0
13. number of truck-loads between  Mumbai  and  Raipur : 30.0
14. number of truck-loads between  Hyderabad  and  Vijayawada : 310.0
15. number of truck-loads between  Hyderabad  and  Raipur : 60.0

Total cost of transportation is **2380570.0**


---


As we can see that when the Lucknow-Simla link was not disrupted 175 units were transported from Lucknow to Simla which was the available stock at Lucknow warehouse and 95 units were transported from Delhi warehouse but now as the route is disrupted, demand of Simla is satisfied by Mumbai (90 units) and Delhi (180 units) which have large transportation cost thus increasing the total cost of transportation from 2192870.0 to 2380570.0

**Exercise 1.10**
If the Kolkata-Dibrugarh link and Delhi-Raipur is disrupted along with Lucknow-Simla link and no transportation is possible along these links, to solve the optimization model without changing the pyomo model we should increase the cost of transportation for these link to a very high amount as this would result in the model not taking up these route for calculating the optimal value since our objective is to minimize the transportation cost and including this link in the model will result in very high cost.

In [ ]:
df.iloc[4,6]=16000
df.iloc[3,7]=17000

In [ ]:
model.new_objective.deactivate()
model.updated_objective=Objective(expr=sum(model.x[i,j]*df.iloc[i,j+1] for i in row_indices for j in col_indices),sense=minimize)

In [ ]:
opt_cbc = SolverFactory('cbc')
result = opt_cbc.solve(model)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [ ]:
#display solution
print('\nObjective = ', model.updated_objective())

print('\nDecision Variables')
for i in row_indices:
  for j in col_indices:
    if(model.x[i,j].value != 0):
      print('number of truck-loads between ',df.iloc[i,0],' and ', df.columns[j+1],':', model.x[i,j].value)

print('\nConstraints')
model.constraint.display()


Objective =  2518200.0

Decision Variables
number of truck-loads between  Ahmedabad  and  Jodhpur : 300.0
number of truck-loads between  Ahmedabad  and  Dibrugarh : 50.0
number of truck-loads between  Bengaluru  and  Trichy : 120.0
number of truck-loads between  Bengaluru  and  Vijayawada : 65.0
number of truck-loads between  Chennai  and  Trichy : 305.0
number of truck-loads between  Delhi  and  Simla : 180.0
number of truck-loads between  Kolkata  and  Raipur : 95.0
number of truck-loads between  Kolkata  and  Agartala : 110.0
number of truck-loads between  Lucknow  and  Dibrugarh : 175.0
number of truck-loads between  Mumbai  and  Hassan : 205.0
number of truck-loads between  Mumbai  and  Simla : 90.0
number of truck-loads between  Mumbai  and  Raipur : 30.0
number of truck-loads between  Hyderabad  and  Vijayawada : 310.0
number of truck-loads between  Hyderabad  and  Raipur : 60.0

Constraints
constraint : Size=16
    Key : Lower : Body  : Upper
      1 :  None : 350.0 : 350.0
  

**Exercise 1.11**
The new solution values are:
1. number of truck-loads between  Ahmedabad  and  Jodhpur : 300.0
2. number of truck-loads between  Ahmedabad  and  Dibrugarh : 50.0
3. number of truck-loads between  Bengaluru  and  Trichy : 120.0
4. number of truck-loads between  Bengaluru  and  Vijayawada : 65.0
5. number of truck-loads between  Chennai  and  Trichy : 305.0
6. number of truck-loads between  Delhi  and  Simla : 180.0
7. number of truck-loads between  Kolkata  and  Raipur : 95.0
8. number of truck-loads between  Kolkata  and  Agartala : 110.0
9. number of truck-loads between  Lucknow  and  Dibrugarh : 175.0
10. number of truck-loads between  Mumbai  and  Hassan : 205.0
11. number of truck-loads between  Mumbai  and  Simla : 90.0
12. number of truck-loads between  Mumbai  and  Raipur : 30.0
13. number of truck-loads between  Hyderabad  and  Vijayawada : 310.0
14. number of truck-loads between  Hyderabad  and  Raipur : 60.0

The new cost of transportation is **2518200.0**


---



As the Kolkata-Dibrugarh route  is disrupted the previous 95 units which were transported from Kolkata to Dibrugarh are now transported by Ahmedabad and Lucknow in addition to the previous units (45,85) which they were transporting already thus increasing the total cost of transportation. 
On the other hand there were no units transported by Delhi to Raipur previously so there is no effect of this disruption in the total cost of transportation but initially Ahmedabad,Mumbai and Hyderabad were transporting (5,120,60) units to Raipur which are now being transported by Kolkata,Mumbai and Hyderabad (95,30,60) units contribute to the increased transportation cost.